In [31]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
pd.options.display.max_columns = 100

In [43]:
#Assigning Path
path_road = r'Z:\Masterfile\1. Shapefile\5. Road\1. All Road'
path_poi = r'Z:\Masterfile\1. Shapefile\4. POI'
path_zmp = r'Z:\000. Processing\REGION I\ZMP'

In [44]:
#Preparing and Reading Files
poi = gpd.read_file(os.path.join(path_poi, 'poi_priority.gpkg'))
road = gpd.read_file(os.path.join(path_road, 'Road_With_Characteristics.gpkg'))
zmp = gpd.read_file(os.path.join(path_zmp, 'ID_ZMP_MASTERFILE_REGION_I.shp'))

In [48]:
#Changing Prjection and creating road length
zmp = zmp.to_crs(epsg = 4326)
road = road.to_crs(epsg = 32748)
road['length'] = road.length
road = road.to_crs(epsg = 4326)

# ZMP Category

In [5]:
#Changing Projection
zmp = zmp.to_crs(epsg = 4326)

In [18]:
#Join zmp with road
zmp_road = gpd.sjoin(road, zmp, how = 'right', predicate = 'intersects')

In [25]:
#Creating rank
zmp_road['rank'] = zmp_road.groupby(['ZMP_CODE'])['length'].rank(ascending = False)

In [33]:
#Filter by ZMP Rank
zmp_road = zmp_road[zmp_road['rank']==1]

In [46]:
#Applying ZMP Category
def zmp_cat(a):
    if a=='>5 m':
        return 'ZMP Commercial'
    else:
        return 'ZMP Non Commercial'
zmp_road['zmp_category'] = zmp_road['Perkiraan Luas'].apply(lambda x: zmp_cat(x))
zmp_road = zmp_road[['ZMP_CODE','zmp_category']]

c:\Users\User\anaconda3\envs\geo_env\lib\site-packages\geopandas\geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


# ZMP With POI Class

In [49]:
#Merging ZMP with POI
zmp_join = gpd.sjoin(poi, zmp, how = 'right', predicate='intersects')

In [51]:
# Create ZMP Summary with POI Class
zmp_dissolve = zmp_join.pivot_table(index = 'ZMP_CODE', values = 'Sub_Class', columns='POI_Class', aggfunc='count').reset_index().fillna(0)

POI_Class,ZMP_CODE,Apartment,CVS,Community Based,Eatery,Education,Finance,Gov,Health Center,Pariwisata,Religion,School,Supermarket,Traditional Market,Transportation,Workshop
0,3171010_001,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3171010_002,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3171010_003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3171010_004,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3171010_005,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [53]:
zmp_final.columns

Index(['OBJECTID_1', 'OBJECTID_2', 'OBJECTID', 'CEN_LNG', 'CEN_LAT',
       'GOOGLE_MAP', 'PROVINCE', 'KECAMATAN', 'CITY', 'ZMP_CODE', 'ZMP_NAME',
       'zmp_area_s', 'adjusted_m', 'prioritize', 'app_area_n', 'app_area_i',
       'landmark', 'Score_UID', 'Score_Spen', 'Score_Visi', 'Score_Velo',
       'delta_time', 'count_uid', 'average_sp', 'total_velo', 'total_visi',
       'average_ve', 'average_vi', 'av_visit_1', 'score_vi_1', 'POI_Gov',
       'POI_CVS', 'POI_Transp', 'POI_School', 'POI_Market', 'POI_Religi',
       'POI_Eatery', 'POI_Health', 'Score_POI', 'Population', 'Regional_I',
       'City_Type', 'Score_City', 'Editor_PIC', 'layer', 'path', 'Source',
       'Shape_Leng', 'Shape_Le_1', 'Shape_Area', 'geometry', 'zmp_category',
       'Apartment', 'CVS', 'Community Based', 'Eatery', 'Education', 'Finance',
       'Gov', 'Health Center', 'Pariwisata', 'Religion', 'School',
       'Supermarket', 'Traditional Market', 'Transportation', 'Workshop'],
      dtype='object')

In [58]:
#Finalizing ZMP
zmp_final = zmp.merge(zmp_road, how = 'left', on = 'ZMP_CODE').merge(zmp_dissolve, how ='left', on = 'ZMP_CODE')
zmp_final = zmp_final[['PROVINCE', 'KECAMATAN', 'CITY', 'ZMP_CODE', 'ZMP_NAME','zmp_category',
       'Apartment', 'CVS', 'Community Based', 'Eatery', 'Education', 'Finance',
       'Gov', 'Health Center', 'Pariwisata', 'Religion', 'School',
       'Supermarket', 'Traditional Market', 'Transportation', 'Workshop','geometry']]
zmp_final.to_file(r'Z:\Masterfile\1. Shapefile\8. ZMP\zmp_masterfile.gpkg')